In [2]:
import os
import re
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv("../data/data.csv")
print(data.shape)
data[:3]

(601, 15)


,ID,RegionVariable,EXP,H1_0,H1_1,H1_2,H1_3,H2_0,H2_1,H2_2,H2_3,H3_0,H3_1,H3_2,H3_3
0,442135,2,Experienced,1295.0,135.0,523.0,346.0,NaN,240.0,792.0,1407.0,NaN,NaN,305.0,NaN
1,995668,4,Newbie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-160.0,330.0,205.0,765.0
2,675738,5,Newbie,1095.0,135.0,323.0,646.0,818.0,140.0,192.0,807.0,-310.0,130.0,5.0,165.0


#### Long to wide

In [6]:
feature_models = [col for col in data.columns if re.match("H[0-9]_[0-9]",col) is not None]
feature_models

['H1_0',
 'H1_1',
 'H1_2',
 'H1_3',
 'H2_0',
 'H2_1',
 'H2_2',
 'H2_3',
 'H3_0',
 'H3_1',
 'H3_2',
 'H3_3']

In [7]:
features = list(set([ re.sub("_[0-9]","",feature_model) for feature_model in feature_models]))
features

['H1', 'H2', 'H3']

In [12]:
long_df = pd.wide_to_long(data,i=['ID','RegionVariable','EXP'],j='ModelID',stubnames=features,sep="_")
long_df[:10]

H1      H2     H3
ID     RegionVariable EXP         ModelID                       
442135 2              Experienced 0        1295.0     NaN    NaN
                                  1         135.0   240.0    NaN
                                  2         523.0   792.0  305.0
                                  3         346.0  1407.0    NaN
995668 4              Newbie      0           NaN     NaN -160.0
                                  1           NaN     NaN  330.0
                                  2           NaN     NaN  205.0
                                  3           NaN     NaN  765.0
675738 5              Newbie      0        1095.0   818.0 -310.0
                                  1         135.0   140.0  130.0

#### Wide to long

In [9]:
data2 = long_df.reset_index()
data2[:10]

,ID,RegionVariable,EXP,ModelID,H1,H2,H3
0,442135,2,Experienced,0,1295.0,NaN,NaN
1,442135,2,Experienced,1,135.0,240.0,NaN
2,442135,2,Experienced,2,523.0,792.0,305.0
3,442135,2,Experienced,3,346.0,1407.0,NaN
4,995668,4,Newbie,0,NaN,NaN,-160.0
5,995668,4,Newbie,1,NaN,NaN,330.0
6,995668,4,Newbie,2,NaN,NaN,205.0
7,995668,4,Newbie,3,NaN,NaN,765.0
8,675738,5,Newbie,0,1095.0,818.0,-310.0
9,675738,5,Newbie,1,135.0,140.0,130.0


In [11]:
data2.set_index(['ID','RegionVariable','EXP','ModelID']).unstack(level=3)[:10]

H1                             H2  \
ModelID                                0      1       2       3       0   
ID    RegionVariable EXP                                                  
2627  1              Newbie       1245.0 -115.0    23.0    46.0  1318.0   
7219  2              Newbie        895.0 -365.0   523.0   346.0   318.0   
8640  2              Experienced   795.0 -165.0   -77.0   246.0   218.0   
9355  2              Experienced   795.0  135.0     NaN     NaN   418.0   
9619  3              Newbie       1095.0  235.0   523.0   846.0   618.0   
9770  3              Newbie        845.0   35.0   223.0   646.0   318.0   
9958  3              Experienced  1245.0  385.0  1023.0  1346.0   518.0   
12662 1              Experienced   995.0  135.0   523.0   846.0   418.0   
13622 5              Experienced  1295.0  435.0     NaN     NaN   818.0   
14237 5              Experienced  1095.0  235.0     NaN     NaN  1018.0   

                                                           H3                \
ModelID                               1      2       3      0      1      2   
ID    RegionVariable EXP                                                      
2627  1              Newbie       -10.0  292.0   607.0 -260.0   30.0  -95.0   
7219  2              Newbie      -260.0  392.0   907.0 -210.0  130.0  -95.0   
8640  2              Experienced -160.0   -8.0   607.0 -410.0  -70.0 -195.0   
9355  2              Experienced  240.0  792.0  1407.0 -110.0  430.0  105.0   
9619  3              Newbie       140.0  192.0   807.0 -210.0  130.0    5.0   
9770  3              Newbie        40.0  292.0  1007.0 -210.0   30.0    5.0   
9958  3              Experienced  240.0  792.0  1407.0    NaN  230.0  105.0   
12662 1              Experienced   40.0  392.0  1107.0 -210.0  180.0   55.0   
13622 5              Experienced  240.0    NaN  2007.0 -160.0  180.0    NaN   
14237 5              Experienced  340.0    NaN  1607.0 -110.0  230.0    5.0   

                                         
ModelID                               3  
ID    RegionVariable EXP                 
2627  1              Newbie       165.0  
7219  2              Newbie       165.0  
8640  2              Experienced  -35.0  
9355  2              Experienced    NaN  
9619  3              Newbie       365.0  
9770  3              Newbie       365.0  
9958  3              Experienced  565.0  
12662 1              Experienced  165.0  
13622 5              Experienced  565.0  
14237 5              Experienced  365.0

#### Other functions to customize multi-level indices
    set_index() / reset_index()
    stack() / unstack()
    swaplevel()
    .T

In [13]:
# Comprehensive EG
# Note: the default count with groupby ignores NAs, while len is simple row count for each group that would include NAs
agg = long_df.reset_index().groupby(['RegionVariable','EXP'])[features].agg({'count':len,'mean':np.mean})
agg.columns = pd.Index(agg.columns).str.join("_")
agg.unstack().T.swaplevel().sort_index().T

NameError: name 'stub_names' is not defined